# This notebook will be used for Segmentation and Clustering of Neighborhood in Toronto


<B>Part 1</B>

Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.

To create the dataframe:

1.The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood.<BR>
2.Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.<BR>
3.More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that - M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the - neighborhoods separated with a comma as shown in row 11 in the above table.<BR>
4.If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.<BR>
5.Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.<BR>
6.In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.


<B>Importing Libraries Pandas, Requests and BeautifulSoup</B>

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

Scraping the list of Postal Codes from https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [2]:
L_url = ' https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(L_url).text

In [3]:
soup = BeautifulSoup(source,'xml')

In [4]:
table = soup.find('table')

In [5]:
#Assigning Column Names
column_names = ['PostalCode', 'Borough','Neighborhood']
df = pd.DataFrame(columns = column_names)

In [6]:
#displaying the dataframe column headings
df

,PostalCode,Borough,Neighborhood


In [7]:
#searching for all the postal codes, Boroughs and Neighorhoods and adding it to our dataframe
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

In [8]:
#This is how our Dataframe looks now
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"



Remove Rows where Borough is not assigned

In [9]:
df = df[df.Borough != 'Not assigned']
#after removing not assigned boroughs
df

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [10]:
#Two or more neighborhoods under one Postal Code will be combined under one postal code separated by commas
df = df.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(list).apply(lambda x:', '.join(x)).to_frame().reset_index()

In [11]:
#if neighborhood has not been assigned, it will be same as the borough
for index, row in df.iterrows():
    if row['Neighborhood'] == 'Not assigned':
        row['Neighborhood'] = row['Borough']

# Final Data Frame After Cleaning

In [12]:
df

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [13]:
df.shape

(103, 3)